# Section 1: Data Generation

Create Environment and dataset

In [1]:
import torch
import random
import numpy as np

from simple_stru_sampler import make_generator_params
from rl4co.envs import SDVRPEnv

seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

params = make_generator_params(
    num_loc=18,
    outer_size=(8, 6),
    inner_size=(4, 2),
    vgap_range=(1.0, 2.0),
    hgap_range=(0.3, 0.5),
)

env = SDVRPEnv(generator_params=params)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

########################batch size ###############################
td_init = env.reset(batch_size=100).to(device)


Check dataset

In [2]:
def get_tensor(td, *keys):
    for k in keys:
        if k in td.keys():
            return td[k]
    raise KeyError(f"None of keys {keys} found in tensordict")

coords   = get_tensor(td_init, "locs", "coords") 
capacity = get_tensor(td_init, "capacity", "veh_capacity", "vehicle_capacity")  
demand   = get_tensor(td_init, "demand") * capacity

B, Np1, _ = coords.shape
N = Np1 - 1  

for b in range(B):
    depot_xy  = coords[b, 0, :]          # (2,)
    cities_xy = coords[b, 1:1+N, :]      # (N,2)
    demands_b = demand[b]                # (N,)
    cap_b     = capacity[b].item() if capacity[b].ndim == 0 else float(capacity[b].squeeze().item())
    print(f"\n=== Sample {b+1} ===")
    print(f"vehicle_capacity: {cap_b:.1f}")
    print(f"depot: (x={depot_xy[0]:.4f}, y={depot_xy[1]:.4f})")
    for i in range(N):
        x, y = cities_xy[i]
        d = demands_b[i]
        print(f"city{i+1}: (x={x:.4f}, y={y:.4f}), demand={float(d):.1f}")


=== Sample 1 ===
vehicle_capacity: 100.0
depot: (x=6.1461, y=5.2430)
city1: (x=2.0905, y=3.7354), demand=10.0
city2: (x=2.5679, y=3.7354), demand=5.0
city3: (x=3.0453, y=3.7354), demand=5.0
city4: (x=3.5226, y=3.7354), demand=5.0
city5: (x=4.0000, y=3.7354), demand=10.0
city6: (x=4.4774, y=3.7354), demand=5.0
city7: (x=4.9547, y=3.7354), demand=5.0
city8: (x=5.4321, y=3.7354), demand=5.0
city9: (x=5.9095, y=3.7354), demand=10.0
city10: (x=2.0905, y=2.2646), demand=10.0
city11: (x=2.5679, y=2.2646), demand=5.0
city12: (x=3.0453, y=2.2646), demand=5.0
city13: (x=3.5226, y=2.2646), demand=5.0
city14: (x=4.0000, y=2.2646), demand=10.0
city15: (x=4.4774, y=2.2646), demand=5.0
city16: (x=4.9547, y=2.2646), demand=5.0
city17: (x=5.4321, y=2.2646), demand=5.0
city18: (x=5.9095, y=2.2646), demand=10.0

=== Sample 2 ===
vehicle_capacity: 100.0
depot: (x=1.1602, y=3.7228)
city1: (x=2.3431, y=3.6428), demand=20.0
city2: (x=2.7573, y=3.6428), demand=10.0
city3: (x=3.1715, y=3.6428), demand=10.0
ci

# Section 2: Gurobi

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import time

coords_np   = coords.detach().cpu().numpy()
demand_np   = demand.detach().cpu().numpy()
capacity_np = capacity.detach().cpu().view(-1).numpy()

all_objs = []
all_routes = []

total_start = time.perf_counter()   # Timer of all instances

for b in range(B):
    inst_start = time.perf_counter()   # Timer of one instance

    # ---------- current instance ----------
    loc_b  = coords_np[b]
    dem_b  = demand_np[b]
    Q_b    = float(capacity_np[b])

    n = N

    dist = np.linalg.norm(
        loc_b[:, None, :] - loc_b[None, :, :],
        axis=-1
    )  # shape: [N+1, N+1]

    total_demand = float(dem_b.sum())
    
    max_vehicles = int(np.ceil(total_demand / Q_b))

    # ---------- SDVRP ----------
    m = gp.Model(f"sdvrp_{b}")
    m.Params.OutputFlag = 0

    # x[i,j] = 1
    x = m.addVars(n + 1, n + 1, vtype=GRB.BINARY, name="x")

    # y[i,j] >= 0 
    y = m.addVars(n + 1, n + 1, lb=0.0, vtype=GRB.CONTINUOUS, name="y")

    for i in range(n + 1):
        m.addConstr(x[i, i] == 0, name=f"no_self_x_{i}")
        m.addConstr(y[i, i] == 0, name=f"no_self_y_{i}")

    # objective
    m.setObjective(
        gp.quicksum(
            dist[i, j] * x[i, j]
            for i in range(n + 1) for j in range(n + 1) if i != j
        ),
        GRB.MINIMIZE
    )

    for k in range(n + 1):
        m.addConstr(
            gp.quicksum(x[i, k] for i in range(n + 1) if i != k) ==
            gp.quicksum(x[k, j] for j in range(n + 1) if j != k),
            name=f"deg_{k}"
        )

    # number of outgoing <= max_vehicles
    m.addConstr(
        gp.quicksum(x[0, j] for j in range(1, n + 1)) <= max_vehicles,
        name="max_vehicles"
    )

    # ---------- capacity constraint <= Q * x_ij ----------
    for i in range(n + 1):
        for j in range(n + 1):
            if i != j:
                m.addConstr(
                    y[i, j] <= Q_b * x[i, j],
                    name=f"cap_{i}_{j}"
                )

    # sum_j y[0,j] - sum_i y[i,0] = total_demand
    m.addConstr(
        gp.quicksum(y[0, j] for j in range(1, n + 1)) -
        gp.quicksum(y[i, 0] for i in range(1, n + 1))
        == total_demand,
        name="d_depot"
    )

    # in - out= demand_j
    for j in range(1, n + 1):
        m.addConstr(
            gp.quicksum(y[i, j] for i in range(n + 1) if i != j) -
            gp.quicksum(y[j, k] for k in range(n + 1) if k != j)
            == float(dem_b[j - 1]),
            name=f"d_cust_{j}"
        )

    m.Params.MIPGap = 0.0
    m.optimize()
    
    inst_end = time.perf_counter()   # instance timer end
    inst_elapsed = inst_end - inst_start

    if m.status != GRB.OPTIMAL:
        print(f"\n=== Instance {b} ===")
        print("No optimal solution")
        print(f"Time (build+solve): {inst_elapsed:.4f} s")
        all_objs.append(None)
        all_routes.append(None)
        continue

    all_objs.append(m.objVal)

    routes_b = []

    used = {}
    for i in range(n + 1):
        for j in range(n + 1):
            if i != j:
                used[(i, j)] = False

    for j in range(1, n + 1):
        if x[0, j].X > 0.5 and not used[(0, j)]:
            route = [0, j]
            used[(0, j)] = True
            cur = j

            while True:
                next_candidates = [
                    k for k in range(n + 1)
                    if (cur, k) in used and (not used[(cur, k)]) and x[cur, k].X > 0.5
                ]
                if not next_candidates:
                    break
                nxt = next_candidates[0]
                route.append(nxt)
                used[(cur, nxt)] = True
                cur = nxt
                if cur == 0:
                    break

            routes_b.append(route)

    all_routes.append(routes_b)

    # print
    print(f"\n=== Instance {b} ===")
    print(f"Optimal total distance: {m.objVal:.4f}")
    print("Routes (0 is depot, 1..N are customers):")
    for r_idx, r in enumerate(routes_b):
        print(f"  Route {r_idx+1}: {r}")

    print(f"Solver time (Gurobi Runtime): {m.Runtime:.4f} s")
    print(f"Total time for this instance (build+solve): {inst_elapsed:.4f} s")

# ====== total time ======
# print(all_objs)
total_end = time.perf_counter()
total_elapsed = total_end - total_start
print(f"\nTotal wall-clock time for all {B} instances (build+solve): {total_elapsed:.4f} s")

avg_obj = sum(all_objs) / len(all_objs)
print(f"Average optimal distance over 100 samples: {avg_obj:.4f}")


Set parameter Username
Set parameter LicenseID to value 2747896
Academic license - for non-commercial use only - expires 2026-11-30

=== Instance 0 ===
Optimal total distance: 17.3156
Routes (0 is depot, 1..N are customers):
  Route 1: [0, 9, 8, 7, 0]
  Route 2: [0, 18, 17, 16, 15, 14, 13, 12, 11, 10, 1, 2, 3, 4, 5, 6, 0]
Solver time (Gurobi Runtime): 0.0670 s
Total time for this instance (build+solve): 0.0814 s

=== Instance 1 ===
Optimal total distance: 22.1722
Routes (0 is depot, 1..N are customers):
  Route 1: [0, 1, 2, 3, 0]
  Route 2: [0, 4, 5, 14, 13, 12, 11, 10, 0]
  Route 3: [0, 6, 7, 8, 9, 18, 17, 16, 15, 0]
Solver time (Gurobi Runtime): 0.1980 s
Total time for this instance (build+solve): 0.3614 s

=== Instance 2 ===
Optimal total distance: 14.3639
Routes (0 is depot, 1..N are customers):
  Route 1: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
  Route 2: [0, 18, 17, 16, 15, 14, 13, 12, 11, 10, 0]
Solver time (Gurobi Runtime): 0.1860 s
Total time for this instance (build+solve): 0.1930 

# Section 3: GA

In [ ]:
import math, random, numpy as np
import time

torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

total_start = time.perf_counter() 

def _to_np(x):
    return x.detach().cpu().numpy() if torch.is_tensor(x) else np.asarray(x)

def build_distance_matrix(depot_xy, cities_xy):
    depot_xy  = _to_np(depot_xy).reshape(-1)         # -> (2,)
    cities_xy = _to_np(cities_xy)                    # -> (N,2)
    depot_xy  = np.atleast_2d(depot_xy)              # -> (1,2)
    cities_xy = np.asarray(cities_xy)
    locs = np.vstack([depot_xy, cities_xy])          # (N+1,2)

    n = locs.shape[0]
    D = np.zeros((n, n), dtype=float)
    for i in range(n):
        for j in range(i+1, n):
            d = float(np.linalg.norm(locs[i] - locs[j]))
            D[i, j] = D[j, i] = d
    return D

def split_routes_by_capacity(order, demands, capacity, allow_split=True, eps=1e-9):
    routes, remain, route = [], float(capacity), []
    dem = _to_np(demands).astype(float).copy()
    for cid in order:
        need = dem[cid-1]
        while need > eps:
            if remain >= need + eps:
                route.append((cid, need))
                remain -= need
                need = 0.0
            else:
                if allow_split:
                    if remain > eps:
                        route.append((cid, remain))
                        need -= remain
                    if route:
                        routes.append(route)
                    route, remain = [], float(capacity)
                else:
                    if route:
                        routes.append(route)
                    route, remain = [], float(capacity)
    if route:
        routes.append(route)
    return routes

def route_length(routes, D):
    total = 0.0
    for r in routes:
        prev = 0
        for (cid, _) in r:
            total += D[prev, cid]
            prev = cid
        total += D[prev, 0]
    return total

def ordered_crossover(p1, p2):
    n = len(p1)
    a, b = sorted(random.sample(range(n), 2))
    child = [-1]*n
    child[a:b+1] = p1[a:b+1]
    fill = [g for g in p2 if g not in child]
    j = 0
    for i in range(n):
        if child[i] == -1:
            child[i] = fill[j]; j += 1
    return child

def mutate_swap(order, p=0.2):
    if random.random() < p:
        i, j = sorted(random.sample(range(len(order)), 2))
        order[i], order[j] = order[j], order[i]
    return order

def two_opt_improve(order, D, iters=60):
    best = order[:]
    n = len(best)
    def approx_len(seq):
        prev, tot = 0, 0.0
        for c in seq:
            tot += D[prev, c]; prev = c
        tot += D[prev, 0]
        return tot
    best_len = approx_len(best)
    for _ in range(iters):
        i, j = sorted(random.sample(range(1, n-1), 2))
        cand = best[:i] + list(reversed(best[i:j+1])) + best[j+1:]
        L = approx_len(cand)
        if L < best_len - 1e-12:
            best, best_len = cand, L
    return best

def fitness(order, D, demands, capacity, allow_split):
    routes = split_routes_by_capacity(order, demands, capacity, allow_split)
    return -route_length(routes, D)

def nearest_neighbor_init(D, N):
    unvis = set(range(1, N+1))
    cur = 0
    seq = []
    while unvis:
        nxt = min(unvis, key=lambda c: D[cur, c])
        seq.append(nxt)
        unvis.remove(nxt)
        cur = nxt
    return seq

def solve_ga_for_sample(depot_xy, cities_xy, demands, capacity,
                        pop_size=160, generations=400,
                        cx_rate=0.9, mut_rate=0.2,
                        allow_split=True, seed=123):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    demands = _to_np(demands).astype(float)
    capacity = float(capacity)
    N = len(demands)

    D = build_distance_matrix(depot_xy, cities_xy)
    base = list(range(1, N+1))

    population = []
    for _ in range(pop_size):
        o = base[:]
        random.shuffle(o)
        population.append(two_opt_improve(o, D, iters=20))
    for _ in range(min(12, pop_size//8)):
        population.append(two_opt_improve(nearest_neighbor_init(D, N), D, iters=50))

    best_order, best_fit = None, -1e30
    for _ in range(generations):
        fits = [fitness(ind, D, demands, capacity, allow_split) for ind in population]
        bi = int(np.argmax(fits))
        if fits[bi] > best_fit:
            best_fit = fits[bi]
            best_order = population[bi][:]

        def tournament():
            k = 3
            cand = random.sample(range(len(population)), k)
            return max(cand, key=lambda idx: fits[idx])

        new_pop = []
        while len(new_pop) < pop_size:
            p1 = population[tournament()]
            p2 = population[tournament()]
            c = ordered_crossover(p1, p2) if random.random() < cx_rate else p1[:]
            mutate_swap(c, p=mut_rate)
            c = two_opt_improve(c, D, iters=15)
            new_pop.append(c)
        population = new_pop

    routes = split_routes_by_capacity(best_order, demands, capacity, allow_split)
    total_len = route_length(routes, D)
    return routes, total_len

print("\n=== GA solutions on td_init ===")
allow_split = True

all_ga_costs = []
all_ga_times = [] 
all_ga_routes = []


total_start = time.perf_counter()

for b in range(B):
    inst_start = time.perf_counter()

    depot_xy  = coords[b, 0, :]
    cities_xy = coords[b, 1:1+N, :]
    demands_b = demand[b]
    cap_b     = capacity[b].item() if capacity[b].ndim == 0 else float(capacity[b].squeeze().item())

    routes, cost = solve_ga_for_sample(
        depot_xy, cities_xy, demands_b, cap_b,
        pop_size=160, generations=400,
        allow_split=allow_split, seed=seed
    )

    inst_end = time.perf_counter()     # Time ending after one instance
    inst_elapsed = inst_end - inst_start
    all_ga_costs.append(cost)
    all_ga_times.append(inst_elapsed)
    all_ga_routes.append(routes)   # save all of the routes

    print(f"\nSample {b+1}: capacity={cap_b:.1f}, total_length={cost:.4f}")
    print(f"  Time used for this sample: {inst_elapsed:.4f} s")
    for k, r in enumerate(routes, 1):
        pretty = " -> ".join([f"C{cid}({amt:.1f})" for cid, amt in r])
        print(f"  Route {k}: Depot -> {pretty} -> Depot")

total_end = time.perf_counter()
total_elapsed = total_end - total_start   # Total time

# ======== Print total information ========
print("\n=== All GA total lengths ===")
print(" ".join([f"{c:.4f}" for c in all_ga_costs]))

avg_cost = sum(all_ga_costs) / len(all_ga_costs) if all_ga_costs else float("nan")
avg_time = sum(all_ga_times) / len(all_ga_times) if all_ga_times else float("nan")

print(f"\n=== GA average total length over {len(all_ga_costs)} samples: {avg_cost:.4f} ===")
print(f"=== GA average time per sample: {avg_time:.4f} s ===")
print(f"=== GA total wall-clock time for all {len(all_ga_costs)} samples: {total_elapsed:.4f} s ===")


=== GA solutions on td_init ===

Sample 1: capacity=100.0, total_length=17.3156
  Time used for this sample: 7.0643 s
  Route 1: Depot -> C18(10.0) -> C17(5.0) -> C16(5.0) -> C15(5.0) -> C14(10.0) -> C13(5.0) -> C12(5.0) -> C11(5.0) -> C10(10.0) -> C1(10.0) -> C2(5.0) -> C3(5.0) -> C4(5.0) -> C5(10.0) -> C6(5.0) -> Depot
  Route 2: Depot -> C7(5.0) -> C8(5.0) -> C9(10.0) -> Depot

Sample 2: capacity=100.0, total_length=22.9744
  Time used for this sample: 6.8388 s
  Route 1: Depot -> C11(10.0) -> C12(10.0) -> C13(10.0) -> C14(20.0) -> C15(10.0) -> C16(10.0) -> C17(10.0) -> C18(20.0) -> Depot
  Route 2: Depot -> C9(20.0) -> C8(10.0) -> C7(10.0) -> C6(10.0) -> C5(20.0) -> C4(10.0) -> C3(10.0) -> C2(10.0) -> Depot
  Route 3: Depot -> C1(20.0) -> C10(20.0) -> Depot

Sample 3: capacity=100.0, total_length=15.6899
  Time used for this sample: 6.9932 s
  Route 1: Depot -> C10(16.0) -> C11(8.0) -> C12(8.0) -> C13(8.0) -> C14(16.0) -> C15(8.0) -> C16(8.0) -> C17(8.0) -> C18(16.0) -> C1(4.0) ->

# Section 4: NN

In [ ]:
def solve_nn_for_sample(depot_xy, cities_xy, demands, capacity,
                        allow_split=True):
    demands = _to_np(demands).astype(float)
    capacity = float(capacity)
    N = len(demands)

    D = build_distance_matrix(depot_xy, cities_xy)

    order = nearest_neighbor_init(D, N)  # e.g. [3, 7, 1, 2, ...]

    routes = split_routes_by_capacity(order, demands, capacity, allow_split=allow_split)

    total_len = route_length(routes, D)

    return routes, total_len


In [6]:
print("\n=== Nearest Neighbor (NN) solutions on td_init ===")
allow_split_nn = True

all_nn_costs = []
all_nn_times = []

nn_total_start = time.perf_counter()  

for b in range(B):
    nn_inst_start = time.perf_counter() 

    depot_xy  = coords[b, 0, :]
    cities_xy = coords[b, 1:1+N, :]
    demands_b = demand[b]
    cap_b     = capacity[b].item() if capacity[b].ndim == 0 else float(capacity[b].squeeze().item())

    routes_nn, cost_nn = solve_nn_for_sample(
        depot_xy, cities_xy, demands_b, cap_b,
        allow_split=allow_split_nn
    )

    nn_inst_end = time.perf_counter()
    nn_inst_elapsed = nn_inst_end - nn_inst_start

    all_nn_costs.append(cost_nn)
    all_nn_times.append(nn_inst_elapsed)

    print(f"\n[NN] Sample {b+1}: capacity={cap_b:.1f}, total_length={cost_nn:.4f}")
    print(f"  Time used for this sample (NN): {nn_inst_elapsed:.4f} s")
    for k, r in enumerate(routes_nn, 1):
        pretty = ' -> '.join([f"C{cid}({amt:.1f})" for cid, amt in r])
        print(f"  Route {k}: Depot -> {pretty} -> Depot")

nn_total_end = time.perf_counter()
nn_total_elapsed = nn_total_end - nn_total_start

print("\n=== All NN total lengths ===")
print(" ".join([f"{c:.4f}" for c in all_nn_costs]))

nn_avg_cost = sum(all_nn_costs) / len(all_nn_costs) if all_nn_costs else float("nan")
nn_avg_time = sum(all_nn_times) / len(all_nn_times) if all_nn_times else float("nan")

print(f"\n=== NN average total length over {len(all_nn_costs)} samples: {nn_avg_cost:.4f} ===")
print(f"=== NN average time per sample: {nn_avg_time:.4f} s ===")
print(f"=== NN total wall-clock time for all {len(all_nn_costs)} samples: {nn_total_elapsed:.4f} s ===")



=== Nearest Neighbor (NN) solutions on td_init ===

[NN] Sample 1: capacity=100.0, total_length=19.7671
  Time used for this sample (NN): 0.1018 s
  Route 1: Depot -> C9(10.0) -> C8(5.0) -> C7(5.0) -> C6(5.0) -> C5(10.0) -> C4(5.0) -> C3(5.0) -> C2(5.0) -> C1(10.0) -> C10(10.0) -> C11(5.0) -> C12(5.0) -> C13(5.0) -> C14(10.0) -> C15(5.0) -> Depot
  Route 2: Depot -> C16(5.0) -> C17(5.0) -> C18(10.0) -> Depot

[NN] Sample 2: capacity=100.0, total_length=23.8724
  Time used for this sample (NN): 0.0015 s
  Route 1: Depot -> C1(20.0) -> C2(10.0) -> C3(10.0) -> C4(10.0) -> C5(20.0) -> C6(10.0) -> C7(10.0) -> C8(10.0) -> Depot
  Route 2: Depot -> C9(20.0) -> C18(20.0) -> C17(10.0) -> C16(10.0) -> C15(10.0) -> C14(20.0) -> C13(10.0) -> Depot
  Route 3: Depot -> C12(10.0) -> C11(10.0) -> C10(20.0) -> Depot

[NN] Sample 3: capacity=100.0, total_length=16.4842
  Time used for this sample (NN): 0.0020 s
  Route 1: Depot -> C10(16.0) -> C11(8.0) -> C12(8.0) -> C13(8.0) -> C14(16.0) -> C15(8.0) -